# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load CSV file generated from WeatherPy Folder
weather_data = pd.read_csv("weather_df.csv")
weather_data
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(20)


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,punta arenas,CL,2022-09-13 16:38:10,-53.1500,-70.9167,0.0,86.0,30.578,2.06
1,upernavik,GL,2022-09-13 16:38:10,72.7868,-56.1549,100.0,97.0,40.352,2.18
3,lipari,IT,2022-09-13 16:38:10,38.4674,14.9540,1.0,59.0,77.594,0.04
4,saskylakh,RU,2022-09-13 16:38:11,71.9167,114.0833,100.0,97.0,35.744,1.71
5,upington,ZA,2022-09-13 16:35:17,-28.4478,21.2561,0.0,26.0,54.248,0.51
6,kirakira,SB,2022-09-13 16:38:11,-10.4544,161.9205,100.0,85.0,79.610,3.88
7,ribeira grande,PT,2022-09-13 16:38:12,38.5167,-28.7000,75.0,94.0,74.066,4.12
8,pevek,RU,2022-09-13 16:38:12,69.7008,170.3133,100.0,88.0,35.672,2.73
9,hermanus,ZA,2022-09-13 16:38:12,-34.4187,19.2345,43.0,70.0,58.190,3.10
10,vaini,TO,2022-09-13 16:38:13,-21.2000,-175.2000,20.0,88.0,77.432,3.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,taonan,CN,2022-09-13 16:39:22,45.3333,122.7833,0.0,28.0,73.616,2.72
1,sarakhs,IR,2022-09-13 16:39:42,36.5449,61.1577,0.0,12.0,77.018,2.54
2,saint-pierre,RE,2022-09-13 16:39:28,-21.3393,55.4781,0.0,57.0,76.946,3.09
3,arys,KZ,2022-09-13 16:39:59,42.4252,68.7950,0.0,17.0,78.998,3.68
4,torbat-e jam,IR,2022-09-13 16:40:12,35.2440,60.6225,0.0,13.0,74.696,1.14
5,ampanihy,MG,2022-09-13 16:40:24,-24.7000,44.7500,0.0,43.0,78.422,5.12
6,taloqan,AF,2022-09-13 16:40:25,36.7361,69.5345,0.0,17.0,79.880,2.29
7,catabola,AO,2022-09-13 16:40:26,-12.1500,17.2833,0.0,39.0,74.426,0.65
8,itaueira,BR,2022-09-13 16:40:42,-7.6033,-43.0256,0.0,35.0,75.092,4.22
9,dondo,MZ,2022-09-13 16:40:59,-19.6094,34.7431,0.0,52.0,78.584,4.11


In [6]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,taonan,CN,2022-09-13 16:39:22,45.3333,122.7833,0.0,28.0,73.616,2.72,Makro Hotel
1,sarakhs,IR,2022-09-13 16:39:42,36.5449,61.1577,0.0,12.0,77.018,2.54,مهمان سرای شرکت گاز
2,saint-pierre,RE,2022-09-13 16:39:28,-21.3393,55.4781,0.0,57.0,76.946,3.09,Alize Plage
3,arys,KZ,2022-09-13 16:39:59,42.4252,68.7950,0.0,17.0,78.998,3.68,Gostinitsa
4,torbat-e jam,IR,2022-09-13 16:40:12,35.2440,60.6225,0.0,13.0,74.696,1.14,White building
5,ampanihy,MG,2022-09-13 16:40:24,-24.7000,44.7500,0.0,43.0,78.422,5.12,HOTEL SHARRON AMPANIHY OUEST
6,taloqan,AF,2022-09-13 16:40:25,36.7361,69.5345,0.0,17.0,79.880,2.29,Saraisang street
7,catabola,AO,2022-09-13 16:40:26,-12.1500,17.2833,0.0,39.0,74.426,0.65,Residencial Princesa
8,itaueira,BR,2022-09-13 16:40:42,-7.6033,-43.0256,0.0,35.0,75.092,4.22,Pousada e Restaurante Cheiro e Sabor
9,dondo,MZ,2022-09-13 16:40:59,-19.6094,34.7431,0.0,52.0,78.584,4.11,Pensao Casual


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig


Figure(layout=FigureLayout(height='420px'))